# Demo 2: parameterizing and packing polymer systems
Here we show how to incorporate force field parameters, solvent admolecules, and other multiple polymers into Topology before exporting it to an MD engine 

In [7]:
from pathlib import Path
from polymerist.genutils.fileutils.pathutils import is_empty, assemble_path

from openff.toolkit import Molecule, Topology # this import might take a sec the first time you run this cell


EXAMPLE_DIR = Path('polymer_loading_examples')
assert EXAMPLE_DIR.exists() and not is_empty(EXAMPLE_DIR)

OUTPUT_DIR = Path('scratch') # dummy file for writing without tampering with example inputs
OUTPUT_DIR.mkdir(exist_ok=True)

## Working on individual polymers
For convenience, we've provided a pre-made example SDF for a poly(N-isopropylacrylamide) (PNIPAAm) polymer  
NOTE: if you haven't already, please run through the [prior demo](2.1-topology_and_fragment.ipynb) to see what this file means and how to create your own

In [8]:
from polymerist.mdtools.openfftools.topology import topology_from_sdf, get_largest_offmol

pnipaam_sdf = assemble_path(EXAMPLE_DIR, 'pnipaam', 'sdf')
assert pnipaam_sdf.exists()

pnipaam_top = topology_from_sdf(pnipaam_sdf)
# pnipaam_top.visualize()
pnipaam = get_largest_offmol(pnipaam_top) # a useful trick for working with individual Molecules in a single-mol or solvated topology
pnipaam.visualize(backend='nglview')

NGLWidget()

In [9]:
# residue partition metadata is preserved by the SDF
for atom in pnipaam.atoms[:10]: 
    print(atom.metadata)

{'residue_name': 'pnipaam_TERM1', 'residue_number': 80, 'insertion_code': ' ', 'chain_id': ' ', 'match_info': '{"27": ["pnipaam_MIDDLE", 1], "28": ["pnipaam_MIDDLE", 1], "80": ["pnipaam_TERM1", 1]}', 'substructure_query_id': 1}
{'residue_name': 'pnipaam_TERM1', 'residue_number': 80, 'insertion_code': ' ', 'chain_id': ' ', 'match_info': '{"28": ["pnipaam_MIDDLE", 2], "80": ["pnipaam_TERM1", 2]}', 'substructure_query_id': 2}
{'residue_name': 'pnipaam_TERM1', 'residue_number': 80, 'insertion_code': ' ', 'chain_id': ' ', 'match_info': '{"27": ["pnipaam_MIDDLE", 19], "28": ["pnipaam_MIDDLE", 19], "80": ["pnipaam_TERM1", 19]}', 'substructure_query_id': 19}
{'residue_name': 'pnipaam_TERM1', 'residue_number': 80, 'insertion_code': ' ', 'chain_id': ' ', 'match_info': '{"27": ["pnipaam_MIDDLE", 20], "28": ["pnipaam_MIDDLE", 20], "80": ["pnipaam_TERM1", 20]}', 'substructure_query_id': 20}
{'residue_name': 'pnipaam_TERM1', 'residue_number': 80, 'insertion_code': ' ', 'chain_id': ' ', 'match_info':

### Custom metadata
In principle, there's no limit to the kinds of data you can associate with a polymer, via the Molecule.properties field.  
For example, here we insert patent info about PNIPAAm which you can verify is written after the chemical table in the resulting SDF

In [10]:
from polymerist.mdtools.openfftools.topology import topology_to_sdf


pnipaam.properties['IUPAC name'] = 'poly(N-isopropylacrylamide)'
pnipaam.properties['First synthesized'] = '1956-12-04'
pnipaam.properties['Patent No.'] = 'US-2773063-A'
pnipaam.properties['Patent Holder'] = 'Edward H. Sprecht'

sdf_path_annotated = assemble_path(OUTPUT_DIR, 'pnipaam_annotated', 'sdf')
topology_to_sdf(sdf_path_annotated, pnipaam.to_topology()) # individual Molecule objects must be bundled into a topology before exporting to SDF

### Assigning [atomic partial charges](https://en.wikipedia.org/wiki/Partial_charge) to polymers
This is a niche-but-vital step, as it governs the electrostatic interactions within and without the polymer chains which will eventually be realized in an molecular dynamics engine  
This is a particularly difficult task for macromolecules, though speedy methods such as pretrained graph neural networks (GNNs) or [custom library charges](2.4-RCT_demo.ipynb) are available 

`polymerist` provides user-friendly hooks for various rapid partial charge wrappers (assuming you've installed one of the [parameterization toolkits](https://github.com/timbernat/polymerist?tab=readme-ov-file#2-parameterization-toolkits)).  
These hook cache the resulting partial charge values and charging method directly to the Molecule's metadata, enabling tracking of provenance during simulation preparation


In [11]:
from polymerist.mdtools.openfftools.partialcharge.molchargers import MolCharger
from polymerist.mdtools.openfftools.partialcharge.rescharge import LibraryCharger

MolCharger.subclass_registry # show which partial charge wrappers are available

{'AM1-BCC-ELF10': polymerist.mdtools.openfftools.partialcharge.molchargers.ABE10Charger,
 'Espaloma-AM1-BCC': polymerist.mdtools.openfftools.partialcharge.molchargers.EspalomaCharger,
 'NAGL': polymerist.mdtools.openfftools.partialcharge.molchargers.NAGLCharger,
 'RCT': polymerist.mdtools.openfftools.partialcharge.rescharge.interface.LibraryCharger}

In [12]:
from polymerist.mdtools.openfftools.partialcharge.molchargers import (
    NAGLCharger,  # requires having OpenFF NAGL installed
    ABE10Charger, # requires having OpenEye toolkits installed and licensed
)

# choose your partial charge method here
# charger = ABE10Charger() # NOTE: this method is unworkably slow for anything bigger than ~150 atoms; it is just placed here to indicate you COULD use AM1-BCC if desired

# assign partial charges
print('Partial charges initial:', pnipaam.partial_charges) # to show you nothing is up my sleeve, we verify that the charges are unset prior to the invocation

charger = NAGLCharger()
pnipaam_charged = charger.charge_molecule(pnipaam)
print(pnipaam_charged.properties['charge_method']) # charge method is recorded for provenance
print('Partial charges after charger:')
display(pnipaam_charged.partial_charges)

pnipaam_charged_sdf = assemble_path(OUTPUT_DIR, 'pnipaam_charged', 'sdf')
topology_to_sdf(pnipaam_charged_sdf, pnipaam_charged.to_topology()) # export the charged molecule to SDF

Partial charges initial: None
NAGL
Partial charges after charger:


array([-0.0682364 , -0.16009238,  0.07492361,  0.07492361,  0.67297864,
       -0.60463637, -0.58531231,  0.06106056,  0.10563403,  0.30443016,
       -0.11142188,  0.04263705,  0.04263705,  0.04263705, -0.11142188,
        0.07013664,  0.04263705,  0.04263705,  0.04263705, -0.06131664,
       -0.13314712,  0.06848101,  0.06848101,  0.67696285, -0.60019469,
       -0.58371472,  0.05439134,  0.10460396,  0.30554375, -0.11168082,
        0.04263494,  0.04263494,  0.04263494, -0.11168082,  0.0695611 ,
        0.04263494,  0.04263494,  0.04263494, -0.05900551, -0.1357445 ,
        0.06816921,  0.06816921,  0.67651624, -0.60081243, -0.58519083,
        0.05642067,  0.10463348,  0.30544111, -0.11168082,  0.04263494,
        0.04263494,  0.04263494, -0.11168082,  0.0695611 ,  0.04263494,
        0.04263494,  0.04263494, -0.0590038 , -0.13644057,  0.06816921,
        0.06816921,  0.67644   , -0.60081243, -0.58519083,  0.05623807,
        0.10463348,  0.30544111, -0.11168082,  0.04263494,  0.04263494,
        0.04263494, -0.11168082,  0.0695611 ,  0.04263494,  0.04263494,
        0.04263494, -0.0590038 , -0.13644057,  0.06816921,  0.06816921,
        0.67644   , -0.60081243, -0.58519083,  0.05623807,  0.10463348,
        0.30544111, -0.11168082,  0.04263494,  0.04263494,  0.04263494,
       -0.11168082,  0.0695611 ,  0.04263494,  0.04263494,  0.04263494,
       -0.0590038 , -0.13644057,  0.06816921,  0.06816921,  0.67644   ,
       -0.60081243, -0.58519083,  0.05623807,  0.10463348,  0.30544111,
       -0.11168082,  0.04263494,  0.04263494,  0.04263494, -0.11168082,
        0.0695611 ,  0.04263494,  0.04263494,  0.04263494, -0.0590038 ,
       -0.13644057,  0.06816921,  0.06816921,  0.67644   , -0.60081243,
       -0.58519083,  0.05623807,  0.10463348,  0.30544111, -0.11168082,
        0.04263494,  0.04263494,  0.04263494, -0.11168082,  0.0695611 ,
        0.04263494,  0.04263494,  0.04263494, -0.0590038 , -0.13644057,
        0.06816921,  0.06816921,  0.67644   , -0.60081243, -0.58519083,
        0.05623807,  0.10463348,  0.30544111, -0.11168082,  0.04263494,
        0.04263494,  0.04263494, -0.11168082,  0.0695611 ,  0.04263494,
        0.04263494,  0.04263494, -0.0590038 , -0.13644057,  0.06816921,
        0.06816921,  0.67644   , -0.60081243, -0.58519083,  0.05623807,
        0.10463348,  0.30544111, -0.11168082,  0.04263494,  0.04263494,
        0.04263494, -0.11168082,  0.0695611 ,  0.04263494,  0.04263494,
        0.04263494, -0.0590038 , -0.13644057,  0.06816921,  0.06816921,
        0.67644   , -0.60081243, -0.58519083,  0.05623807,  0.10463348,
        0.30544111, -0.11168082,  0.04263494,  0.04263494,  0.04263494,
       -0.11168082,  0.0695611 ,  0.04263494,  0.04263494,  0.04263494,
       -0.0590038 , -0.13644057,  0.06816921,  0.06816921,  0.67644   ,
       -0.60081243, -0.58519083,  0.05623807,  0.10463348,  0.30544111,
       -0.11168082,  0.04263494,  0.04263494,  0.04263494, -0.11168082,
        0.0695611 ,  0.04263494,  0.04263494,  0.04263494, -0.0590038 ,
       -0.13644057,  0.06816921,  0.06816921,  0.67644   , -0.60081243,
       -0.58519083,  0.05623807,  0.10463348,  0.30544111, -0.11168082,
        0.04263494,  0.04263494,  0.04263494, -0.11168082,  0.0695611 ,
        0.04263494,  0.04263494,  0.04263494, -0.0590038 , -0.13644057,
        0.06816921,  0.06816921,  0.67644   , -0.60081243, -0.58519083,
        0.05623807,  0.10463348,  0.30544111, -0.11168082,  0.04263494,
        0.04263494,  0.04263494, -0.11168082,  0.0695611 ,  0.04263494,
        0.04263494,  0.04263494, -0.0590038 , -0.13644057,  0.06816921,
        0.06816921,  0.67644   , -0.60081243, -0.58519083,  0.05623807,
        0.10463348,  0.30544111, -0.11168082,  0.04263494,  0.04263494,
        0.04263494, -0.11168082,  0.0695611 ,  0.04263494,  0.04263494,
        0.04263494, -0.0590038 , -0.13644057,  0.06816921,  0.06816921,
        0.67644   , -0.60081243, -0.58519083,  0.05623807,  0.10463348,
        0.30544111, -0.11168082,  0.0426

## Including other molecules in your polymer system

### Coexistent polymers
`polymerist` ships with a lattice-based packer which makes it straightforward to "tile" multiple copies of a polymer into a simulation box  
The packer is flexible and allows you to specify where copies of each polymer are placed in space

We demonstrate a simple "3D checkerboard" here with PNIPAAM and a bisphenol A (BPA) that [I prepared earlier](https://tvtropes.org/pmwiki/pmwiki.php/Main/OneIPreparedEarlier), but the possibilities are only limited by your imagination

In [13]:
# load pre-prepared BPA to use as coexistent polymer
bisphenol_sdf = assemble_path(EXAMPLE_DIR, 'bisphenolA', 'sdf')
assert bisphenol_sdf.exists()

bisphenol_top = topology_from_sdf(bisphenol_sdf)
# bisphenol_top.visualize()
bisphenol = get_largest_offmol(bisphenol_top) # a useful trick for working with individual Molecules in a single-mol or solvated topology
bisphenol_charged = charger.charge_molecule(bisphenol) # charge the bisphenol A molecule

bisphenol_charged.visualize(backend='nglview')

NGLWidget()

In [46]:
from itertools import product as cartesian
import numpy as np

from polymerist.mdtools.openfftools.physprops import effective_radius
from polymerist.mdtools.openfftools.topology import topology_from_molecule_onto_lattice


S : int = 5 # number of polymers to place along each axis (i.e., will have SxSxS alternating box of polymers)
lattice_str = f'{S}x{S}x{S}'

integer_lattice = np.array([int_point for int_point in cartesian(range(S), repeat=3)]) # the "3" is because we are in 3 dimensions
is_odd_idx = np.mod(integer_lattice.sum(axis=1), 2).astype(bool) # analogous to the indices of either color in a 3D checkboard
r_eff = max(effective_radius(pnipaam_charged), effective_radius(bisphenol_charged))
lattice_points = (r_eff * integer_lattice).m_as('angstrom')  # scale by larger of effective radii to avoid collisions; strip units while ensuring magnitudes are as Angstroms

pnipaam_top_packed   = topology_from_molecule_onto_lattice(pnipaam_charged, lattice_points[~is_odd_idx])
bisphenol_top_packed = topology_from_molecule_onto_lattice(bisphenol_charged, lattice_points[is_odd_idx])
mixed_polymer_top = pnipaam_top_packed + bisphenol_top_packed

melt_sdf_path = assemble_path(OUTPUT_DIR, f'PNIPAAm-BPA_{lattice_str}_melt', 'sdf')
topology_to_sdf(melt_sdf_path, mixed_polymer_top) # export the mixed polymer topology to SDF

mixed_polymer_top.visualize()

NGLWidget()

### Packing solvents

## Setting periodic box and exporting with Interchange